In [1]:
import anvil.server
import anvil.email
import anvil.tables as tables
import anvil.tables.query as q
from anvil.tables import app_tables
import anvil.server
from datetime import date,timedelta
import pandas as pd
import anvil.media
import numpy as np
import json
import pandas as pd
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail
from io import StringIO
import json

#for uploading csv/excel file
anvil.server.connect("server_S3Q6TEQAXIULZBGXLCAMZ3DQ-WJUWH6ZSQJZ25NP3")
@anvil.server.callable
def import_csv_data(file):
    with anvil.media.TempFile(file) as file_name:       
        if file.content_type == 'text/csv':
            df = pd.read_csv(file_name)
        else:
            df = pd.read_excel(file_name)

        @anvil.server.callable
        def temp():
            ls_col=[]
            ls=df.columns
            for i in range(len(ls)):
                ls_col.append(ls[i])
            return ls_col   
        #df=rename_col(before,after,df)
        
        @anvil.server.callable
        def rename_col(f,e,p,a,c,full,com,d,pan,l,g,t,depart,u,deg,pas,col,link,face,insta,indus,count,stat,zipc,skil,total,buisness,cin,turnover,incorporation,employ,ctc,note,remark,action,segment,sub,middle,last):
            df.rename(columns={str(f):'First_name',str(e):'Email_ID',str(p):'Phone',str(a):'Aadhar_card',
                               str(c):'Contact_type',str(full):'Full_name',str(com):'Company',str(d):'Designation',str(pan):'Pan_card',str(l):'Location',str(g):'Gender',str(t):'Title',str(depart):'Department',str(u):'University',str(deg):'Degree',str(pas):'Passing_Year',str(col):'College',str(link):'Linkedin_url',str(face):'Facebook_url',str(insta):'Instagram_Id',str(indus):'Industry',str(count):'Country',str(stat):'State',str(zipc):'Zip_code',str(skil):'Key_skill',str(total):'Total_exp',str(buisness):'Buisness_year',str(cin):'CIN_no',str(turnover):'Turnover',str(incorporation):'Date_of_incorporation',str(employ):'Employee',str(ctc):'Ctc',str(note):'Notes',str(remark):'Remarks',str(action):'Next_action',str(segment):'Segment',str(sub):'Sub_segment',str(middle):'Middle_name',str(last):'Last_name'},inplace=True)
            
        @anvil.server.callable
        def add_data_file():
            df.fillna('none',inplace=True)
            for d in df.to_dict(orient="records"):
              # d is now a dict of {columnname -> value} for this row
              # We use Python's **kwargs syntax to pass the whole dict as
              # keyword arguments
              anvil.tables.app_tables.contact_info.add_row(**d)

#for not repeating data           
def func2(date):
    data=app_tables.tec.search()
    for i in data:
        if date == i['Date']:
            return True
        
        else:
            continue  
#for the ststistics retrieved from the send grid   
@anvil.server.callable
def global_status():
    sg = SendGridAPIClient(os.environ.get('type the send grid key here'))
    params = {'start_date': '2021-12-01'}
    response = sg.client.mailbox_providers.stats.get(
    query_params=params)
    d=response.body

    s=str(d,'utf-8')

    final_dictionary = json.loads(s)
    a=final_dictionary
    
    
    for i in range(len(a)):
        a=final_dictionary[i]['stats']
        date=final_dictionary[i]['date']
        x=pd.DataFrame(a)['metrics']
        f=pd.DataFrame.from_records(x)
        for d in f.to_dict(orient="records"):
            
              # d is now a dict of {columnname -> value} for this row
              # We use Python's **kwargs syntax to pass the whole dict as
              # keyword arguments
            f2=func2(date)  
            if f2==True:
                pass
            else:
                anvil.tables.app_tables.tec.add_row(**d,Date=date)

#message output data table
def message_output(name,segment,sub_segment,status,email,date,text='none',sequence_id='none'):
    return app_tables.message_output.add_row(Name=name,Segment=segment,Sub_segment=sub_segment,Status=status,Text=text,Sequence_ID=sequence_id,Email=email,Date=date)

#funnel data tabe
@anvil.server.callable
def funnel_creation(segment,sub_segment):
    @anvil.server.callable
    def funnel_mess(seq_ID,message,date):      
        app_tables.funnel.add_row(Sequence_ID=seq_ID,Messages=message,Date=date,Segment=segment,Sub_segment=sub_segment)
        
        return 'Funnel saved sucessfully'

#for not repeating the click event
def func():
    data=app_tables.message_output.search()
    for i in data:
        if str(date.today()) == i['Date']:
            return True
        
        else:
            continue
            
#For sending the email by clicking the button
@anvil.server.callable
def testing():
    f=func()
    if f == True:
        pass
             
    else:
        funnel=app_tables.funnel.search()
        
        for i in funnel:
            if i['Date']!="":
                if i['Date']==str(date.today()):
                    data=app_tables.contact_info.search(Segment=i['Segment'])
                    for j in data:
                        message = Mail(from_email='aryanksingh1994@gmail.com',
                                       to_emails=j['Email_ID'],
                                       subject='Sending with Twilio SendGrid is Fun',
                                       html_content=f'<strong>{i['Messages']}</strong>')
                        sg = SendGridAPIClient(os.environ.get('type send grid api key here'))
                        response = sg.send(message)
                        message_output(j['Full_name'],i['Segment'],i['Sub_segment'],'sent',email=j['Email_ID'],date=str(date.today()))
                        
        
                else:
                    continue

                        
    return 'Email sucessfully sent'

        
anvil.server.wait_forever()

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


KeyboardInterrupt: 